In [1]:
import pandas as pd
import os
import re
import pickle
import spacy
import string
import nltk
from nltk.corpus import stopwords
import numpy as np

In [2]:
df = pd.read_csv('../data/text_df.csv')

In [3]:
df.head()

,folder,extension,filename,absolute_path,size_mb,created_on,last_modified_on,unix_permission,raw_text
0,../data/t5/,.text,000081.text,../data/t5/000081.text,0.013150,2011-02-08 15:37:52,2011-02-08 15:37:52,-rw-r--r--,NaN
1,../data/t5/,.text,000083.text,../data/t5/000083.text,0.007671,2011-02-08 15:37:52,2011-02-08 15:37:52,-rw-r--r--,PUBLIC NOTICE\n FEDERAL COMMUNICATIONS COMMISS...
2,../data/t5/,.text,000086.text,../data/t5/000086.text,0.026270,2011-02-08 15:37:52,2011-02-08 15:37:52,-rw-r--r--,"Time,F-Scale,Location,County,State,Lat,Lon,Com..."
3,../data/t5/,.text,000087.text,../data/t5/000087.text,0.023494,2011-02-08 15:37:52,2011-02-08 15:37:52,-rw-r--r--,**********************************************...
4,../data/t5/,.text,000088.text,../data/t5/000088.text,0.004139,2011-02-08 15:37:52,2011-02-08 15:37:52,-rw-r--r--,76 APPENDIX.\n able...


In [4]:
# filenames without text
files_w_text = df[df['raw_text'].isna()]
pickle.dump(list(files_w_text['filename'].values), open('../data/file_w_text.pkl', 'wb'))
df.drop(index=files_w_text.index, inplace=True)

In [5]:
nlp = spacy.load('en_core_web_lg')

def lemmatizing(text, nlp):
    doc = nlp(text.replace('\n', ' '))
    sent = []

    for token in doc:
        if token.pos_ not in ('PROPN', 'PUNCT', 'NUM', 'SYM'):
            if token.lemma_ != '-PRON-':
                sent.append(token.lemma_)
    
    sent = " ".join(sent).split()
    
    return " ".join(sent)

In [6]:
def stemming(text):
    stemmer = nltk.stem.SnowballStemmer('english', ignore_stopwords=True)
    stemmer.stopwords = stopwords.words('english')
    
    stemmed_words = []
    
    for w in text.split():
        stemmed_words.append(stemmer.stem(w))
        
    text = ' '.join(stemmed_words)
        
    return text

In [85]:
df['lemmatized_text'] = df['raw_text'].apply(lambda row: lemmatizing(row, nlp))

In [87]:
df['stemmed_text'] = df['lemmatized_text'].apply(lambda row: stemming(row))

In [92]:
df.to_csv('../data/lem_stem_text.csv', index=False)